<a href="https://colab.research.google.com/github/arunsechergy/TSAI-DeepNLP-END/blob/main/assignments/assignment7/Exp3_Sentiment_Analysis_using_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Summary of Experiment 3

> **Analysis**: The below model is overfitting the train and has poor validation accuracy.

**Next immediate step**: will try bidirectional for the model to learn both the ways.

  - Done this, no improvement.
  
**Next second step**: Will try good initialization strategy, that is pretrained embeddings and check - done this, no improvement

Probably not enough data, will add more data by data augmentation



# 0 TorchText

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
mkdir drive/MyDrive/data

In [3]:
cd drive/MyDrive/data

/content/drive/MyDrive/data


In [ ]:
%%shell
  wget http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip

--2020-12-09 10:26:06--  http://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip [following]
--2020-12-09 10:26:07--  https://nlp.stanford.edu/~socherr/stanfordSentimentTreebank.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6372817 (6.1M) [application/zip]
Saving to: ‘stanfordSentimentTreebank.zip’

stanfordSentimentTr 100%[===================>]   6.08M  1.91MB/s    in 3.9s    

2020-12-09 10:26:11 (1.56 MB/s) - ‘stanfordSentimentTreebank.zip’ saved [6372817/6372817]



In [ ]:
%%shell
unzip stanfordSentimentTreebank.zip

Archive:  stanfordSentimentTreebank.zip
   creating: stanfordSentimentTreebank/
  inflating: stanfordSentimentTreebank/datasetSentences.txt  
   creating: __MACOSX/
   creating: __MACOSX/stanfordSentimentTreebank/
  inflating: __MACOSX/stanfordSentimentTreebank/._datasetSentences.txt  
  inflating: stanfordSentimentTreebank/datasetSplit.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._datasetSplit.txt  
  inflating: stanfordSentimentTreebank/dictionary.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._dictionary.txt  
  inflating: stanfordSentimentTreebank/original_rt_snippets.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._original_rt_snippets.txt  
  inflating: stanfordSentimentTreebank/README.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._README.txt  
  inflating: stanfordSentimentTreebank/sentiment_labels.txt  
  inflating: __MACOSX/stanfordSentimentTreebank/._sentiment_labels.txt  
  inflating: stanfordSentimentTreebank/SOStr.txt  
  inflating: stanfo

In [ ]:
ls

drive/  sample_data/


In [4]:
cd ../../..

/content


In [5]:
DATA_DIR = 'drive/MyDrive/data/stanfordSentimentTreebank'

In [6]:
import os
os.path.join(DATA_DIR, 'datasetSentences.txt')

'drive/MyDrive/data/stanfordSentimentTreebank/datasetSentences.txt'

In [7]:
import pandas as pd
datasetSentences = pd.read_csv(os.path.join(DATA_DIR, 'datasetSentences.txt'), sep='\t')

In [8]:
datasetSplit = pd.read_csv(os.path.join(DATA_DIR, 'datasetSplit.txt'), sep=',')
datasetSplit.head()

,sentence_index,splitset_label
0,1,1
1,2,1
2,3,2
3,4,2
4,5,2


In [9]:
splitset_label = {1:'train',2:'test',3:'dev'}

In [10]:
datasetSentences.shape, datasetSplit.shape

((11855, 2), (11855, 2))

In [11]:
datasetSentences.columns, datasetSplit.columns

(Index(['sentence_index', 'sentence'], dtype='object'),
 Index(['sentence_index', 'splitset_label'], dtype='object'))

In [12]:
merged_dataset = datasetSentences.merge(datasetSplit, on='sentence_index')

In [13]:
merged_dataset.shape

(11855, 3)

In [14]:
merged_dataset.head()

,sentence_index,sentence,splitset_label
0,1,The Rock is destined to be the 21st Century 's...,1
1,2,The gorgeously elaborate continuation of `` Th...,1
2,3,Effective but too-tepid biopic,2
3,4,If you sometimes like to go to the movies to h...,2
4,5,"Emerges as something rare , an issue movie tha...",2


add sentiment label to the merged_dataset

In [15]:
# read sentiment_labels text
sentiment_label = pd.read_csv(os.path.join(DATA_DIR, "sentiment_labels.txt"),sep='|')

In [16]:
sentiment_label.shape

(239232, 2)

In [17]:
sentiment_label.head()

,phrase ids,sentiment values
0,0,0.50000
1,1,0.50000
2,2,0.44444
3,3,0.50000
4,4,0.42708


In [18]:
dictionary = pd.read_csv(os.path.join(DATA_DIR, "dictionary.txt"), sep='|',header=None)
dictionary.head()

,0,1
0,!,0
1,! ',22935
2,! '',18235
3,! Alas,179257
4,! Brilliant,22936


In [19]:
dictionary.columns = ['phrases', 'phrase_id']

In [20]:
dictionary.head()

,phrases,phrase_id
0,!,0
1,! ',22935
2,! '',18235
3,! Alas,179257
4,! Brilliant,22936


In [21]:
dictionary.shape

(239232, 2)

In [ ]:
org_txt =pd.read_csv(os.path.join(DATA_DIR, "original_rt_snippets.txt"),   sep='\t')
org_txt.head()

,"The Rock is destined to be the 21st Century's new ``Conan'' and that he's going to make a splash even greater than Arnold Schwarzenegger, Jean-Claud Van Damme or Steven Segal."
0,The gorgeously elaborate continuation of ``The...
1,Effective but too-tepid biopic
2,If you sometimes like to go to the movies to h...
3,"Emerges as something rare, an issue movie that..."
4,The film provides some great insight into the ...


In [22]:
dataset_with_phrase_id = merged_dataset.merge(dictionary, left_on='sentence', right_on ='phrases', how='left')

In [23]:

sentiment_label.head()

,phrase ids,sentiment values
0,0,0.50000
1,1,0.50000
2,2,0.44444
3,3,0.50000
4,4,0.42708


In [24]:
ads = dataset_with_phrase_id.merge(sentiment_label, left_on='phrase_id', right_on='phrase ids', how='left')

In [25]:
ads.shape

(11855, 7)

In [26]:
ads.columns

Index(['sentence_index', 'sentence', 'splitset_label', 'phrases', 'phrase_id',
       'phrase ids', 'sentiment values'],
      dtype='object')

In [27]:
# ads.drop(['phrases', 'phrase ids'], axis=1, inplace=True)
ads['labels'] = [(1 if 0 <=i <=0.2 else (2 if 0.2<i<=0.4 else (3 if 0.4 <i<=0.6 else (4 if 0.6 <i<=0.8 else 5)))) for i in ads['sentiment values'] ]
ads.rename(columns={'sentence':'tweets'}, inplace=True)

In [33]:
# DATA Augmentation approaches
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
import random
nltk.download('stopwords')
nltk.download('punkt')

stoplist  = set(stopwords.words('english'))

def remove_stopwords(text):
    return [word for word in word_tokenize(text) if not word in stoplist]
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [319]:
final_df = ads[['tweets','labels', 'splitset_label']].copy()

In [320]:
final_df.shape

(11855, 3)

In [321]:
train_df, test_df, dev_df = [groups for name, groups in final_df.groupby('splitset_label')]

In [322]:
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
dev_df.reset_index(drop=True, inplace=True)

In [323]:
train_df.head()

,tweets,labels,splitset_label
0,The Rock is destined to be the 21st Century 's...,4,1
1,The gorgeously elaborate continuation of `` Th...,5,1
2,Singer\/composer Bryan Adams contributes a sle...,4,1
3,You 'd think by now America would have had eno...,3,1
4,Yet the act is still charming here .,4,1


In [402]:
# experimenting with one, random_insertion
ran_swapped_tweets = random_swap(train_df['tweets'], 5)

new_df = pd.DataFrame()
new_df['tweets'] = ran_swapped_tweets
new_df['labels'] = train_df['labels']
#new_df taken from below
new_df['splitset_label'] = train_df['splitset_label']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [406]:
new_df.head()

,tweets,labels,splitset_label
0,The Rock is destined to be the 21st Century 's...,4,1
1,The gorgeously elaborate continuation of `` Th...,5,1
2,Singer\/composer Bryan Adams contributes a sle...,4,1
3,You 'd think by now America would have had eno...,3,1
4,Yet the act is still charming here .,4,1


In [405]:
train_df.shape, new_df.shape

((8544, 3), (8544, 3))

In [407]:
augmented_train_df = pd.concat([train_df[['tweets','labels','splitset_label']], new_df], ignore_index=True)

In [408]:
print("train_data count", augmented_train_df.labels.value_counts())

train_data count 4    4432
2    4184
5    3314
3    3098
1    2060
Name: labels, dtype: int64


In [324]:
print("original\n", ads.labels.value_counts())
print("augmented if any\n", final_df.labels.value_counts())

original
 2    2971
4    2966
5    2342
3    2144
1    1432
Name: labels, dtype: int64
augmented if any
 2    2971
4    2966
5    2342
3    2144
1    1432
Name: labels, dtype: int64


## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [409]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [410]:
Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [411]:
fields = [('tweets', Tweet),('labels',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [412]:
final_df.head()

,tweets,labels,splitset_label
0,The Rock is destined to be the 21st Century 's...,4,1
1,The gorgeously elaborate continuation of `` Th...,5,1
2,Effective but too-tepid biopic,3,2
3,If you sometimes like to go to the movies to h...,4,2
4,"Emerges as something rare , an issue movie tha...",5,2


In [417]:

df = final_df[['tweets','labels']].copy()
train_df = augmented_train_df.copy()

In [418]:
train_example = [data.Example.fromlist([train_df.tweets[i],train_df.labels[i]], fields) for i in range(train_df.shape[0])] 
test_example = [data.Example.fromlist([test_df.tweets[i],test_df.labels[i]], fields) for i in range(test_df.shape[0])] 
valid_example = [data.Example.fromlist([dev_df.tweets[i],dev_df.labels[i]], fields) for i in range(dev_df.shape[0])] 

In [419]:
train = data.Dataset(train_example, fields)
valid = data.Dataset(valid_example, fields)
test = data.Dataset(test_example, fields)

In [420]:
(len(train), len(valid), len(test))

(17088, 1101, 2210)

An example from the dataset:

In [421]:
vars(train.examples[10])

{'labels': 5,
 'tweets': ['Good',
  'fun',
  ',',
  'good',
  'action',
  ',',
  'good',
  'acting',
  ',',
  'good',
  'dialogue',
  ',',
  'good',
  'pace',
  ',',
  'good',
  'cinematography',
  '.']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [422]:
MAX_VOCAB_SIZE = 25_000

Tweet.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [423]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  17203
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 16082), (',', 14264), ('the', 12172), ('and', 8942), ('of', 8886), ('a', 8842), ('to', 6052), ('-', 5468), ("'s", 5086), ('is', 5082)]
Labels :  defaultdict(<function _default_unk_index at 0x7fd2d6300ea0>, {4: 0, 2: 1, 5: 2, 3: 3, 1: 4})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [424]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [425]:
device

device(type='cuda')

In [426]:
batch_size = 64

In [427]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = batch_size, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

In [428]:
train_iterator, test_iterator = data.BucketIterator.splits((train, test), batch_size = batch_size, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [429]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [430]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout, pad_idx, bidirectional=False):
        
        super().__init__()          
        
        self.bidirectional = bidirectional
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           bidirectional=bidirectional,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.dropout = nn.Dropout(dropout)
        scaling_factor = 2 if bidirectional else 1
        
        self.fc = nn.Linear(hidden_dim*scaling_factor, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.dropout(self.embedding(text))
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        if self.bidirectional:
          hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
          hidden = self.dropout(hidden)
          hidden = hidden[0]
          
        dense_outputs = self.fc(hidden)   
        # Final activation function softmax
        output = F.softmax(dense_outputs, dim=1)
            
        return output

In [431]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 256
num_output_nodes = 5
num_layers = 2
dropout = 0.5
bidirectional=True
PAD_IDX = Tweet.vocab.stoi[Tweet.pad_token]


# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout, pad_idx=PAD_IDX, bidirectional=bidirectional)

In [432]:
PAD_IDX = Tweet.vocab.stoi[Tweet.pad_token]

UNK_IDX = Tweet.vocab.stoi[Tweet.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[PAD_IDX] = torch.zeros(embedding_dim)

In [433]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(17203, 300, padding_idx=1)
  (encoder): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=5, bias=True)
)
The model has 7,883,209 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [434]:
import torch.optim as optim

# define optimizer and loss
# lr=0.005
# lr = 2e-4
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# criterion = nn.BCELoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    _, predictions = torch.max(preds, 1)
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [435]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        #acc = binary_accuracy(predictions, batch.labels)   
        acc = categorical_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [436]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            # acc = binary_accuracy(predictions, batch.labels)
            acc = categorical_accuracy(predictions, batch.labels)   
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [437]:
N_EPOCHS = 15
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.585 | Train Acc: 28.36%
	 Val. Loss: 1.661 |  Val. Acc: 20.63% 

	Train Loss: 1.587 | Train Acc: 29.20%
	 Val. Loss: 1.536 |  Val. Acc: 34.78% 

	Train Loss: 1.482 | Train Acc: 40.85%
	 Val. Loss: 1.505 |  Val. Acc: 38.25% 

	Train Loss: 1.446 | Train Acc: 44.66%
	 Val. Loss: 1.521 |  Val. Acc: 36.60% 

	Train Loss: 1.423 | Train Acc: 47.19%
	 Val. Loss: 1.516 |  Val. Acc: 37.90% 

	Train Loss: 1.396 | Train Acc: 49.80%
	 Val. Loss: 1.494 |  Val. Acc: 39.73% 

	Train Loss: 1.367 | Train Acc: 52.72%
	 Val. Loss: 1.493 |  Val. Acc: 40.33% 

	Train Loss: 1.342 | Train Acc: 55.31%
	 Val. Loss: 1.490 |  Val. Acc: 40.16% 

	Train Loss: 1.337 | Train Acc: 55.93%
	 Val. Loss: 1.506 |  Val. Acc: 38.25% 

	Train Loss: 1.311 | Train Acc: 58.58%
	 Val. Loss: 1.498 |  Val. Acc: 39.29% 

	Train Loss: 1.290 | Train Acc: 60.93%
	 Val. Loss: 1.496 |  Val. Acc: 39.72% 

	Train Loss: 1.273 | Train Acc: 62.49%
	 Val. Loss: 1.513 |  Val. Acc: 38.33% 

	Train Loss: 1.256 | Train Acc: 64.30%
	

## Model Testing

In [392]:
model.load_state_dict(torch.load('saved_weights.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.540 | Test Acc: 35.03%


In [393]:
path = './sentiment_analysis_saved_weights_1.pt'

In [ ]:
#load weights and tokenizer

path = './sentiment_analysis_saved_weights_1.pt'
# path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    # labels: very negative, negative, neutral, positive, very positive
    categories = {1: "very negative", 2:"negative", 3:"neutral", 4:'positive', 5: 'very positive'}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'very negative'

## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [ ]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [ ]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [ ]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library googletrans for this purpose. 

In [ ]:
import random
import googletrans
import googletrans.Translator

translator = Translator()
sentence = ['The dog slept on the rug']

available_langs = list(googletrans.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, dest=trans_lang) 
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
en_text = [t.text for t in translations_en_random]
print(en_text)